# Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
from scipy.stats import chi2_contingency
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedKFold

from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout
from keras.optimizers import Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# Import Data

In [2]:
data = pd.read_csv('fashion_data_on_Instagram.csv')
data.set_index('UserId', inplace = True)
data.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Clean Data

### Let's See if there are any NaNs in the data

In [3]:
hashtag = data.copy()
hashtag['Hashtags'] = data['Hashtags'].isna()
hashtag.groupby('Hashtags')['MediaCount'].count()

Hashtags
False    24606
True       146
Name: MediaCount, dtype: int64

In [4]:
Caption = data.copy()
Caption['Caption'] = data['Caption'].isna()
Caption.groupby('Caption')['MediaCount'].count()

Caption
False    24717
True        35
Name: MediaCount, dtype: int64

In [5]:
set(data['Comments '].isna())

{False}

##### So there exist NaN values in Hashtags and Captions. There are no other columns that contain NaN.

In [6]:
data.isna().groupby('Hashtags')['Followings'].count()

Hashtags
False    24606
True       146
Name: Followings, dtype: int64

##### There are 146 NaN rows in hashtags. It is possible that this is simply due to the fact that some posts do not have hashtags.

In [7]:
data.isna().groupby('Caption')['Followings'].count()

Caption
False    24717
True        35
Name: Followings, dtype: int64

##### There are 35 NaN rows in Caption

### Maybe the caption or hashtag information is missing because there is either only a caption or only hashtags....

In [8]:
hashtag_nan = list(data['Hashtags'].isna())
caption_nan = list(data['Caption'].isna())

In [9]:
count = 0
for i in range(len(hashtag_nan)):
    if hashtag_nan[i] and caption_nan[i]:
        count += 1
print('The number of times both hashtag_nan and caption_nan is true is', count)

The number of times both hashtag_nan and caption_nan is true is 18


##### So we have missing hashtag both 18/35 times that we have a missing caption. So it is not exactly correlated. Maybe we can just drop the rows that have missing caption or hashtag if we need to use these, or claim that there was not hashtag or caption for these rows.

In [10]:
chi2_contingency([[hashtag_nan.count(True), hashtag_nan.count(False)], [caption_nan.count(True), caption_nan.count(False)]])

(67.09615038449334,
 2.585828482081104e-16,
 1,
 array([[   90.5, 24661.5],
        [   90.5, 24661.5]]))

### I want to try and see if I can make brand category numeric

In [11]:
brand_category = pd.get_dummies(data['BrandCategory'])

In [12]:
brand_category.head()

,Designer,High street,Mega couture,Small couture
UserId,,,,
1171579752855683619_212070047,0,1,0,0
1171594777274371222_176762322,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171508187966229230_2797323089,0,1,0,0


In [13]:
data_new = pd.concat([data, brand_category], sort = False, axis = 1)
data_new.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0


##### Now the brand category is represented in the last four columns of the dataframe with one_hot_encoding.

# Model with Numeric Data Only

First, lets try to build a model with only the numeric data. This will be easy to do and will allow us to understand whether any of these numeric factors influence the number of likes/comments a user gets. 

### Likes Model

In [14]:
X = np.array(data_new[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']])
y = np.array(data_new['Likes'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [16]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [17]:
X_train, X_test = standardize(X_train, X_test)

In [18]:
epsilon = 1e-200
y_train = np.log(y_train + epsilon)
y_test = np.log(y_test + epsilon)

Let's use a Lasso Regression model to regress based on the likes that we receive. I want to use lasso because lasso will send the unimportant variables to zero allowing us to see which features are important. 

In [19]:
lasso = Lasso(alpha = 0.1, max_iter = 10000).fit(X_train, y_train)

In [20]:
predictions = lasso.predict((X_test))
r2_score(y_test, predictions)

0.02953513081793302

Maybe ridge regression will work better?

In [21]:
ridge = Ridge(alpha = 0.1).fit(X_train, y_train)

In [22]:
predictions = ridge.predict((X_test))
r2_score(y_test, predictions)

0.028789646105978117

### Try with stratified crossvalidation

In [23]:
cv = StratifiedKFold(5)
lasso = Lasso(alpha = 0.1, max_iter = 10000)
for train, test in cv.split(X, y):
    X_train, X_test = standardize(X[train], X[test])
    epsilon = 1e-200
    y_train = np.log(y[train] + epsilon)
    y_test = np.log(y[test] + epsilon)
    
    lasso.fit(X_train, y_train)
    predictions = lasso.predict((X_test))
    print(r2_score(y_test, predictions))

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


0.03333041980194418
0.017322227191164608
0.023494040402613048
0.006043123352006852
0.030156451036641663


In [24]:
cv = StratifiedKFold(5)
ridge = Ridge(alpha = 0.1)
for train, test in cv.split(X, y):
    X_train, X_test = standardize(X[train], X[test])
    epsilon = 1e-200
    y_train = np.log(y[train] + epsilon)
    y_test = np.log(y[test] + epsilon)
    
    ridge.fit(X_train, y_train)
    predictions = lasso.predict((X_test))
    print(r2_score(y_test, predictions))

0.03355581371362881
0.023278613663796577
0.02701250716408221
0.014130073519360353
0.030156451036641663


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


### I want to try evening out the amount of data in different categories based on the image features to reduce bias.

In [62]:
data = data_new[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor']]

In [63]:
selfie, body, marketing, product, nonfash, total = 0,0,0,0,0,0
selfiel, bodyl, marketingl, productl, nonfashl = [],[],[],[],[]
thresh = 0.75
for index, row in data.iterrows():
    if row['Selfie'] > thresh:
        selfie+=1
        selfiel.append(index)
    if row['BodySnap'] > thresh:
        body+=1
        bodyl.append(index)
    if row['Marketing'] > thresh:
        marketing+=1
        marketingl.append(index)
    if row['ProductOnly'] > thresh:
        product+=1
        productl.append(index)
    if row['NonFashion'] > thresh:
        nonfash+=1
        nonfashl.append(index)
    total+=1

print('Selfie: ', selfie/total * 100)
print('BodySnap: ', body/total * 100)
print('Marketing: ', marketing/total * 100)
print('ProductOnly: ', product/total * 100)
print('NonFashion: ', nonfash/total * 100)

Selfie:  1.3574660633484164
BodySnap:  22.551712992889463
Marketing:  5.2521008403361344
ProductOnly:  34.68810601163542
NonFashion:  7.558985132514544


### Try with the data that we increase the number of certain kinds of data

In [64]:
X = data_new[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_new[['Likes']]

In [65]:
# add some random data to even out bias
import random
selfie_lst = selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel
indices = random.sample(selfie_lst, selfie*20)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
marketing_lst = marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl
indices = random.sample(marketing_lst, marketing*5)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
nonfash_lst = nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl
indices = random.sample(nonfash_lst, nonfash*2)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])

In [29]:
selfien, bodyn, marketingn, productn, nonfashn, totaln = 0,0,0,0,0,0
thresh = 0.75
for index, row in X.iterrows():
    if row['Selfie'] > thresh:
        selfien+=1
    if row['BodySnap'] > thresh:
        bodyn+=1
    if row['Marketing'] > thresh:
        marketingn+=1
    if row['ProductOnly'] > thresh:
        productn+=1
    if row['NonFashion'] > thresh:
        nonfashn+=1
    totaln+=1

print('Selfie: ', selfien/totaln * 100)
print('BodySnap: ', bodyn/totaln * 100)
print('Marketing: ', marketingn/totaln * 100)
print('ProductOnly: ', productn/totaln * 100)
print('NonFashion: ', nonfashn/totaln * 100)

Selfie:  16.795201371036846
BodySnap:  13.458059602018471
Marketing:  18.930305626963726
ProductOnly:  20.541749976197277
NonFashion:  13.834142625916405


In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [67]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [68]:
X_train, X_test = standardize(X_train, X_test)

In [69]:
epsilon = 1e-200
y_train = np.log(y_train + epsilon)
y_test = np.log(y_test + epsilon)

Let's use a Lasso Regression model to regress based on the likes that we receive. I want to use lasso because lasso will send the unimportant variables to zero allowing us to see which features are important. 

In [70]:
lasso = Lasso(alpha = 0.01, max_iter = 10000).fit(X_train, y_train)

In [71]:
predictions = lasso.predict((X_test))
r2_score(y_test, predictions)

0.03093719280741125

### Including things we know likes are dependent on 

In [21]:
X = data_new[['Followings', 'Followers', 'MediaCount','Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_new['Likes']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [42]:
X_train, X_test = standardize(X_train, X_test)

Let's use a Lasso Regression model to regress based on the likes that we receive. I want to use lasso because lasso will send the unimportant variables to zero allowing us to see which features are important. 

In [43]:
lasso = Lasso(alpha = 0.1, max_iter = 10000).fit(X_train, y_train)

In [44]:
predictions = lasso.predict((X_test))
r2_score(y_test, predictions)

0.2984377062815089

Maybe ridge regression will work better?

In [45]:
ridge = Ridge(alpha = 0.1).fit(X_train, y_train)

In [46]:
predictions = ridge.predict((X_test))
r2_score(y_test, predictions)

0.2984484171726812

So ridge regression is really no different...

# Try a Deep Learning Model

In [52]:
X = np.array(data_new[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']])
y = np.array(data_new['Likes'])

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [54]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [55]:
X_train, X_test = standardize(X_train, X_test)

In [56]:
epsilon = 1e-200
y_train = np.log(y_train + epsilon)
y_test = np.log(y_test + epsilon)

In [57]:
def our_model():
    model = Sequential()

    model.add(Dense(units=64, activation='relu', input_dim = 24))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

In [58]:
estimator = KerasRegressor(build_fn=our_model, epochs=100, batch_size=5, verbose=1)

In [59]:
estimator.fit(X_train, y_train)

Epoch 1/100
22276/22276 [==============================] - 4s 162us/step - loss: 3601.6181 - accuracy: 0.0013
Epoch 2/100
22276/22276 [==============================] - 3s 151us/step - loss: 3545.4630 - accuracy: 0.0012
Epoch 3/100
22276/22276 [==============================] - 4s 158us/step - loss: 3531.0701 - accuracy: 0.0014
Epoch 4/100
22276/22276 [==============================] - 4s 158us/step - loss: 3522.8881 - accuracy: 0.0012
Epoch 5/100
22276/22276 [==============================] - 4s 168us/step - loss: 3505.4012 - accuracy: 9.8761e-04
Epoch 6/100
22276/22276 [==============================] - 3s 153us/step - loss: 3491.3866 - accuracy: 8.5294e-04
Epoch 7/100
22276/22276 [==============================] - 3s 151us/step - loss: 3488.2103 - accuracy: 8.5294e-04
Epoch 8/100
22276/22276 [==============================] - 3s 148us/step - loss: 3462.8254 - accuracy: 0.0012
Epoch 9/100
22276/22276 [==============================] - 3s 150us/step - loss: 3455.4558 - accuracy: 0.001

In [60]:
preds = estimator.predict(X_test)

2476/2476 [==============================] - 0s 41us/step


In [61]:
r2_score(y_test, preds)

0.035906537244450254

### This model with the bias correction method from before

In [72]:
X = data_new[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_new[['Likes']]

In [73]:
# add some random data to even out bias
import random
selfie_lst = selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel
indices = random.sample(selfie_lst, selfie*20)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
marketing_lst = marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl
indices = random.sample(marketing_lst, marketing*5)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
nonfash_lst = nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl
indices = random.sample(nonfash_lst, nonfash*2)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [75]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [76]:
X_train, X_test = standardize(X_train, X_test)

In [77]:
epsilon = 1e-200
y_train = np.log(y_train + epsilon)
y_test = np.log(y_test + epsilon)

In [78]:
estimator = KerasRegressor(build_fn=our_model, epochs=100, batch_size=5, verbose=1)

In [79]:
estimator.fit(X_train, y_train)

Epoch 1/100
37842/37842 [==============================] - 6s 158us/step - loss: 3272.6849 - accuracy: 8.4562e-04
Epoch 2/100
37842/37842 [==============================] - 6s 155us/step - loss: 3112.9580 - accuracy: 0.0010
Epoch 3/100
37842/37842 [==============================] - 6s 156us/step - loss: 2947.3009 - accuracy: 6.8707e-04
Epoch 4/100
37842/37842 [==============================] - 6s 155us/step - loss: 2806.1533 - accuracy: 6.8707e-04
Epoch 5/100
37842/37842 [==============================] - 6s 155us/step - loss: 2732.8819 - accuracy: 7.9277e-04
Epoch 6/100
37842/37842 [==============================] - 6s 156us/step - loss: 2646.2221 - accuracy: 8.4562e-04
Epoch 7/100
37842/37842 [==============================] - 6s 156us/step - loss: 2616.2420 - accuracy: 9.5132e-04
Epoch 8/100
37842/37842 [==============================] - 6s 156us/step - loss: 2557.5132 - accuracy: 4.2281e-04
Epoch 9/100
37842/37842 [==============================] - 6s 157us/step - loss: 2530.7974 -

In [80]:
preds = estimator.predict(X_test)

4205/4205 [==============================] - 0s 35us/step


In [81]:
r2_score(y_test, preds)

0.23445506750890832

Maybe we need to investigate how the input space was normalized. Maybe the categorical variables that are represented by 1's and 0's at the end of the dataframe need to be normalized?

After log normalization of the likes we have worse results on this model.

Average loss value for Keras deep learning model based on just numeric values and predicting likes is 154K.

Maybe try using likes / followers?

# Likes to Follower Ratio

First check if there are any null likes or follower entries

In [50]:
data_new['Likes'].isnull().any() 

False

In [51]:
data_new['Followers'].isnull().any()

False

In [57]:
data_ver2 = pd.concat([data, brand_category], sort = False, axis = 1)
data_ver2['LikesToFollowers'] = data_ver2['Likes'] / data_ver2['Followers']
data_ver2.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,Designer,High street,Mega couture,Small couture,LikesToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0,0,1,0,0,0.009858
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.040870
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.150413
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.155372
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.019737


In [58]:
data_ver2.replace([np.inf, -np.inf], np.nan, inplace = True);

In [59]:
data_test = data_ver2[data_ver2['LikesToFollowers'].isnull()]
data_test.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,Designer,High street,Mega couture,Small couture,LikesToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171240019544303946_23457617,0.0,0.0,1,acnestudios,Designer,"shop, anndemeulemeester, style, acnestudios, t...",Black? Or dark black... @00000 #totokaelo #l...,https://scontent.cdninstagram.com/t51.2885-15/...,37,0,1453842712,https://www.instagram.com/p/BBBFJefqOlK/,0.000158,0.001617,0.007038,0.997941,0.001827,0.000604,0.099600,0.073979,0.000380,0.001582,0.192986,3.91808,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1,0,0,0,1,0,0,0,NaN
1171142669094830425_252313279,36.0,0.0,0,acnestudios,Designer,"acnestudios, collegeface",#collegeface in college\U0001f610 #acnestudios,https://scontent.cdninstagram.com/t51.2885-15/...,18,1,1453831107,https://www.instagram.com/p/BBAvA10s0FZ/,0.870760,0.200230,0.063385,0.003886,0.016565,0.991469,0.052578,0.024295,0.510374,0.032716,1.752410,2.34389,3.308620e-04,0.003890,4.890000e-05,4.975480e-04,0.000018,0.829768,0.150697,0.014749,1,0,0,0,1,0,0,0,NaN
1171278619918133139_23638768,0.0,0.0,0,acnestudios,Designer,NaN,1.26 | \u2603 Cc: \U0001f4f7 @00000,https://scontent.cdninstagram.com/t51.2885-15/...,136,9,1453847313,https://www.instagram.com/p/BBBN7L5OueT/,0.008489,0.907810,0.258318,0.002872,0.000588,0.974532,0.053198,0.020144,0.215500,0.081668,1.268000,5.73313,3.104320e-04,0.000025,1.300000e-05,3.810000e-06,0.011196,0.955701,0.032682,0.000069,1,0,0,0,1,0,0,0,NaN
1170503373243750859_2795691477,0.0,0.0,1,acnestudios,Designer,"sorpreseinmacchina, acnestudios, withstyle, ga...",#jackolsen #jauria #withstyle #migraineprobl...,https://scontent.cdninstagram.com/t51.2885-15/...,10,0,1453754897,https://www.instagram.com/p/BA-dp3KlxnL/,0.375648,0.528273,0.055469,0.001185,0.009081,0.996861,0.134891,0.024669,0.513444,0.315825,1.413850,4.22136,2.080000e-07,0.000181,8.390000e-07,4.160000e-09,0.979905,0.019908,0.000003,0.000001,1,0,0,0,1,0,0,0,NaN
1170007750938083500_2120361935,39.0,0.0,0,alexanderwang,Designer,"jimmychoo, slimthickwithyourcuteass, alexander...","u\""\""I swear she's so perfect she makes me s...",https://scontent.cdninstagram.com/t51.2885-15/...,74,0,1453695814,https://www.instagram.com/p/BA8s9m7HkCs/,0.002184,0.999626,0.000747,0.000168,0.000336,0.771275,0.026620,0.001609,0.331313,0.013736,0.667217,4.49539,8.573980e-04,0.001655,4.457260e-04,2.339453e-03,0.008483,0.939227,0.021567,0.025426,1,0,0,0,1,0,0,0,NaN


Drop rows where LikesToFollowers is NAN. Likes and Followers count at both 0.

In [60]:
data_ver2 = data_ver2.dropna()

In [61]:
data_ver2['LikesToFollowers'].isnull().any()

False

In [62]:
X = data_ver2[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_ver2['LikesToFollowers']

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [64]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [65]:
X_train, X_test = standardize(X_train, X_test)

In [66]:
epsilon = 1e-200
y_train = np.log(y_train+epsilon)
y_test = np.log(y_test+epsilon)

In [67]:
import keras
keras.backend.floatx()
keras.backend.set_floatx('float64')
keras.backend.floatx()

'float64'

In [68]:
def our_model_ver2():
    model = Sequential()
    model.add(Dense(units=128, input_dim = 24))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))
    
    model.add(Dense(units=64, activation = 'tanh'))
    
    model.add(Dense(units=1))
    #opt = Adadelta(clipnorm=0.3)
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasRegressor(build_fn=our_model_ver2, epochs=100, batch_size=500, verbose=1)

# kfold = KFold(n_splits=5)
# results = cross_val_score(estimator, X_train, y_train, cv=kfold)
# print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [70]:
estimator.fit(X_train, y_train)

ValueError: Error when checking input: expected dense_9_input to have shape (24,) but got array with shape (28,)

In [108]:
preds = estimator.predict(X_test)

2455/2455 [==============================] - 0s 16us/step


In [109]:
r2_score(y_test, preds)

0.08561252991158974

When using log normalization of likes/followers we got 0.036 and when we did log normalization of just likes we got 0.035. Additionally, reducing the amount of test data and the batch size improves the results further to 0.0856.

# Try modifying bias and running the deep learning likes to followers model

In [111]:
data = data_ver2[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor']]

In [112]:
selfie, body, marketing, product, nonfash, total = 0,0,0,0,0,0
selfiel, bodyl, marketingl, productl, nonfashl = [],[],[],[],[]
thresh = 0.75
for index, row in data.iterrows():
    if row['Selfie'] > thresh:
        selfie+=1
        selfiel.append(index)
    if row['BodySnap'] > thresh:
        body+=1
        bodyl.append(index)
    if row['Marketing'] > thresh:
        marketing+=1
        marketingl.append(index)
    if row['ProductOnly'] > thresh:
        product+=1
        productl.append(index)
    if row['NonFashion'] > thresh:
        nonfash+=1
        nonfashl.append(index)
    total+=1

print('Selfie: ', selfie/total * 100)
print('BodySnap: ', body/total * 100)
print('Marketing: ', marketing/total * 100)
print('ProductOnly: ', product/total * 100)
print('NonFashion: ', nonfash/total * 100)

Selfie:  1.3486533838569041
BodySnap:  22.450393187466894
Marketing:  5.272379089760828
ProductOnly:  34.82459357046816
NonFashion:  7.554088742207554


In [113]:
X = data_ver2[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_ver2[['LikesToFollowers']]

In [114]:
# add some random selfie data
import random
selfie_lst = selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel
indices = random.sample(selfie_lst, selfie*20)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
marketing_lst = marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl
indices = random.sample(marketing_lst, marketing*5)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
nonfash_lst = nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl
indices = random.sample(nonfash_lst, nonfash*2)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [116]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [117]:
X_train, X_test = standardize(X_train, X_test)

In [118]:
epsilon = 1e-200
y_train = np.log(y_train+epsilon)
y_test = np.log(y_test+epsilon)

In [119]:
def our_model_ver2():
    model = Sequential()
    model.add(Dense(units=128, input_dim = 28))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))
    
    model.add(Dense(units=64, activation = 'tanh'))
    
    model.add(Dense(units=1))
    #opt = Adadelta(clipnorm=0.3)
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasRegressor(build_fn=our_model_ver2, epochs=100, batch_size=500, verbose=1)
print(X_train.shape)
print(y_train.shape)

(37514, 28)
(37514, 1)


In [120]:
estimator.fit(X_train, y_train)

Epoch 1/100
37514/37514 [==============================] - 2s 47us/step - loss: 3524.7731 - accuracy: 0.0000e+00
Epoch 2/100
37514/37514 [==============================] - 1s 38us/step - loss: 3472.0162 - accuracy: 0.0000e+00
Epoch 3/100
37514/37514 [==============================] - 1s 38us/step - loss: 3445.9645 - accuracy: 0.0000e+00
Epoch 4/100
37514/37514 [==============================] - 1s 38us/step - loss: 3425.3117 - accuracy: 0.0000e+00
Epoch 5/100
37514/37514 [==============================] - 2s 42us/step - loss: 3405.4863 - accuracy: 0.0000e+00
Epoch 6/100
37514/37514 [==============================] - 1s 36us/step - loss: 3389.1560 - accuracy: 0.0000e+00
Epoch 7/100
37514/37514 [==============================] - 2s 43us/step - loss: 3383.1850 - accuracy: 0.0000e+00
Epoch 8/100
37514/37514 [==============================] - 2s 46us/step - loss: 3366.8562 - accuracy: 0.0000e+00
Epoch 9/100
37514/37514 [==============================] - 1s 37us/step - loss: 3355.5082 - accu

In [121]:
preds = estimator.predict(X_test)
print(preds)

4169/4169 [==============================] - 0s 68us/step
[ -4.0489634   -4.04917075  -4.04897819 ...  -4.04896317 -10.61378816
 -25.29348649]


In [122]:
r2_score(y_test, preds)

0.3251639724142804

# Try Engagement (Likes + Comments)

In [91]:
data_new['Comments ']

UserId
1171579752855683619_212070047     0
1171594777274371222_176762322     0
1171407552643586413_581125501     3
1171407552643586413_581125501     3
1171508187966229230_2797323089    1
                                 ..
1171575731978099465_389885373     0
1171566570370586806_253427645     1
1171566525099211846_1433754542    0
1171566525099211846_1433754542    0
1171565778125281443_253427645     1
Name: Comments , Length: 24752, dtype: int64

In [134]:
data_ver3 = pd.concat([data_new, brand_category], sort = False, axis = 1)
data_ver3['Engagement'] = data_ver3['Likes'] + data_ver3['Comments ']
data_ver3['EngagementToFollowers'] = data_ver3['Engagement'] / data_ver3['Followers']
# data_ver3.dropna()
data_ver3.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,Designer,High street,Mega couture,Small couture,Engagement,EngagementToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0,0,1,0,0,97,0.009858
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,94,0.040870
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,94,0.155372
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,97,0.160331
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,10,0.021930


In [135]:
data_ver3.replace([np.inf, -np.inf], np.nan, inplace = True);

In [136]:
data_test = data_ver3[data_ver3['EngagementToFollowers'].isnull()]
data_test.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,Designer,High street,Mega couture,Small couture,Engagement,EngagementToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171240019544303946_23457617,0.0,0.0,1,acnestudios,Designer,"shop, anndemeulemeester, style, acnestudios, t...",Black? Or dark black... @00000 #totokaelo #l...,https://scontent.cdninstagram.com/t51.2885-15/...,37,0,1453842712,https://www.instagram.com/p/BBBFJefqOlK/,0.000158,0.001617,0.007038,0.997941,0.001827,0.000604,0.099600,0.073979,0.000380,0.001582,0.192986,3.91808,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1,0,0,0,1,0,0,0,37,NaN
1171142669094830425_252313279,36.0,0.0,0,acnestudios,Designer,"acnestudios, collegeface",#collegeface in college\U0001f610 #acnestudios,https://scontent.cdninstagram.com/t51.2885-15/...,18,1,1453831107,https://www.instagram.com/p/BBAvA10s0FZ/,0.870760,0.200230,0.063385,0.003886,0.016565,0.991469,0.052578,0.024295,0.510374,0.032716,1.752410,2.34389,3.308620e-04,0.003890,4.890000e-05,4.975480e-04,0.000018,0.829768,0.150697,0.014749,1,0,0,0,1,0,0,0,19,NaN
1171278619918133139_23638768,0.0,0.0,0,acnestudios,Designer,NaN,1.26 | \u2603 Cc: \U0001f4f7 @00000,https://scontent.cdninstagram.com/t51.2885-15/...,136,9,1453847313,https://www.instagram.com/p/BBBN7L5OueT/,0.008489,0.907810,0.258318,0.002872,0.000588,0.974532,0.053198,0.020144,0.215500,0.081668,1.268000,5.73313,3.104320e-04,0.000025,1.300000e-05,3.810000e-06,0.011196,0.955701,0.032682,0.000069,1,0,0,0,1,0,0,0,145,NaN
1170503373243750859_2795691477,0.0,0.0,1,acnestudios,Designer,"sorpreseinmacchina, acnestudios, withstyle, ga...",#jackolsen #jauria #withstyle #migraineprobl...,https://scontent.cdninstagram.com/t51.2885-15/...,10,0,1453754897,https://www.instagram.com/p/BA-dp3KlxnL/,0.375648,0.528273,0.055469,0.001185,0.009081,0.996861,0.134891,0.024669,0.513444,0.315825,1.413850,4.22136,2.080000e-07,0.000181,8.390000e-07,4.160000e-09,0.979905,0.019908,0.000003,0.000001,1,0,0,0,1,0,0,0,10,NaN
1170007750938083500_2120361935,39.0,0.0,0,alexanderwang,Designer,"jimmychoo, slimthickwithyourcuteass, alexander...","u\""\""I swear she's so perfect she makes me s...",https://scontent.cdninstagram.com/t51.2885-15/...,74,0,1453695814,https://www.instagram.com/p/BA8s9m7HkCs/,0.002184,0.999626,0.000747,0.000168,0.000336,0.771275,0.026620,0.001609,0.331313,0.013736,0.667217,4.49539,8.573980e-04,0.001655,4.457260e-04,2.339453e-03,0.008483,0.939227,0.021567,0.025426,1,0,0,0,1,0,0,0,74,NaN


In [137]:
data_ver3 = data_ver3.dropna()

In [139]:
data_ver3['EngagementToFollowers'].isnull().any()

False

### Include the bias modification

In [140]:
data = data_ver3[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor']]

In [141]:
selfie, body, marketing, product, nonfash, total = 0,0,0,0,0,0
selfiel, bodyl, marketingl, productl, nonfashl = [],[],[],[],[]
thresh = 0.75
for index, row in data.iterrows():
    if row['Selfie'] > thresh:
        selfie+=1
        selfiel.append(index)
    if row['BodySnap'] > thresh:
        body+=1
        bodyl.append(index)
    if row['Marketing'] > thresh:
        marketing+=1
        marketingl.append(index)
    if row['ProductOnly'] > thresh:
        product+=1
        productl.append(index)
    if row['NonFashion'] > thresh:
        nonfash+=1
        nonfashl.append(index)
    total+=1

print('Selfie: ', selfie/total * 100)
print('BodySnap: ', body/total * 100)
print('Marketing: ', marketing/total * 100)
print('ProductOnly: ', product/total * 100)
print('NonFashion: ', nonfash/total * 100)

Selfie:  1.3486533838569041
BodySnap:  22.450393187466894
Marketing:  5.272379089760828
ProductOnly:  34.82459357046816
NonFashion:  7.554088742207554


In [142]:
X = data_ver3[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_ver3[['EngagementToFollowers']]

In [143]:
# add some random selfie data
import random
selfie_lst = selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel+selfiel
indices = random.sample(selfie_lst, selfie*20)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
marketing_lst = marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl+marketingl
indices = random.sample(marketing_lst, marketing*5)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])
nonfash_lst = nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl+nonfashl
indices = random.sample(nonfash_lst, nonfash*2)
X = X.append(X.loc[indices])
y = y.append(y.loc[indices])

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [145]:
X_train, X_test = standardize(X_train, X_test) # standardize

In [146]:
epsilon = 1e-200
y_train = np.log(y_train+epsilon)
y_test = np.log(y_test+epsilon)

In [147]:
def our_model_ver2():
    model = Sequential()
    model.add(Dense(units=128, input_dim = 28))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))
    
    model.add(Dense(units=64, activation = 'tanh'))
    
    model.add(Dense(units=1))
    #opt = Adadelta(clipnorm=0.3)
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasRegressor(build_fn=our_model_ver2, epochs=100, batch_size=500, verbose=1)
print(X_train.shape)
print(y_train.shape) # run v2 of model

(37485, 28)
(37485, 1)


In [148]:
estimator.fit(X_train, y_train)

Epoch 1/100
37485/37485 [==============================] - 2s 48us/step - loss: 3325.4203 - accuracy: 0.0000e+00
Epoch 2/100
37485/37485 [==============================] - 1s 35us/step - loss: 3272.3351 - accuracy: 0.0000e+00
Epoch 3/100
37485/37485 [==============================] - 1s 36us/step - loss: 3245.4750 - accuracy: 0.0000e+00
Epoch 4/100
37485/37485 [==============================] - 1s 36us/step - loss: 3225.6797 - accuracy: 0.0000e+00
Epoch 5/100
37485/37485 [==============================] - 1s 36us/step - loss: 3207.1236 - accuracy: 0.0000e+00
Epoch 6/100
37485/37485 [==============================] - 1s 36us/step - loss: 3197.0493 - accuracy: 0.0000e+00
Epoch 7/100
37485/37485 [==============================] - 2s 43us/step - loss: 3182.2528 - accuracy: 0.0000e+00
Epoch 8/100
37485/37485 [==============================] - 2s 41us/step - loss: 3167.3085 - accuracy: 0.0000e+00
Epoch 9/100
37485/37485 [==============================] - 1s 40us/step - loss: 3153.6185 - accu

37485/37485 [==============================] - 1s 34us/step - loss: 2485.6638 - accuracy: 0.0000e+00
Epoch 72/100
37485/37485 [==============================] - 1s 36us/step - loss: 2473.7833 - accuracy: 2.6677e-05
Epoch 73/100
37485/37485 [==============================] - 1s 37us/step - loss: 2506.2111 - accuracy: 0.0000e+00
Epoch 74/100
37485/37485 [==============================] - 1s 34us/step - loss: 2468.8190 - accuracy: 0.0000e+00
Epoch 75/100
37485/37485 [==============================] - 1s 33us/step - loss: 2466.4022 - accuracy: 0.0000e+00
Epoch 76/100
37485/37485 [==============================] - 1s 35us/step - loss: 2423.8615 - accuracy: 0.0000e+00
Epoch 77/100
37485/37485 [==============================] - 1s 36us/step - loss: 2431.4948 - accuracy: 0.0000e+00
Epoch 78/100
37485/37485 [==============================] - 1s 34us/step - loss: 2457.0720 - accuracy: 0.0000e+00
Epoch 79/100
37485/37485 [==============================] - 1s 37us/step - loss: 2408.8804 - accuracy

In [149]:
preds = estimator.predict(X_test)
print(preds)

4166/4166 [==============================] - 0s 44us/step
[-26.03353119 -29.99559724  -3.81717428 ...  -9.19417147 -13.37482277
  -5.14241917]


In [150]:
r2_score(y_test, preds) # score for engagement to followers

0.282758386939426

only a slight improvement from LikesToFollowers deep learning model